In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install GPUtil
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#!pip install wandb
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import pandas as pd




import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 14.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
     |████████████████████████████████| 596 kB 75.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 93.9 MB/s 
     |████████████████████████████████| 212 kB 95.5 MB/s 
     |████████████████████████████████| 141 kB 91.0 MB/s 
     |████████████████████████████████| 127 kB 86.4 MB/s 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, 

ModuleNotFoundError: ignored

In [ ]:

def ConvertLabel2ModelLabel(label):
  if label == 'Positive':
    o = 2
  elif label == 'Neutral':
    o = 1
  elif label == 'Negative':
    o = 0
  elif label == 'spam':
    o = 1
  else: 
    print(" error at ConvertLabel2ModelLabel label is " , label)
    o = np.nan
  return o

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [ ]:
train_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/train_dataset.csv'
valid_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/valid_dataset.csv'
#model_name = "Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment"
#out_dir = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/tomas23_3:4:4/with_high'
#out_dir2 = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/tomas23_3:4:4/with_high_then_low'

import shutil





name_of_text_column = "text"
hyperparameter_tuning = False
reduce_ = False
num_hyper_trails = 100
epoch = 60

df_train = pd.read_csv(train_path)

df_train_high = df_train[df_train['quality']=="high"]
df_train_low = df_train[df_train['quality']=="low"]

df_valid = pd.read_csv(valid_path)

display(df_train)
display(df_valid)
display(df_train_high)
display(df_train_low)

In [ ]:
df_train['label'] = df_train['label'].apply(ConvertLabel2ModelLabel)
df_valid['label'] = df_valid['label'].apply(ConvertLabel2ModelLabel)

df_train_high['label'] = df_train_high['label'].apply(ConvertLabel2ModelLabel)
df_train_low['label'] = df_train_low['label'].apply(ConvertLabel2ModelLabel)

display(df_train)
display(df_valid)
display(df_train_high)
display(df_train_low)


In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
#learning_rate = 5e-05
#batch_size = 16
#eval_batch_size = 4
seed = 40
#optimizer = Adam 
#with betas=(0.9,0.999) and epsilon=1e-08
adam_beta1 = 0.9
adam_beta2 =0.999
lr_scheduler_type = "linear"
num_epochs = 15
#args = TrainingArguments("test_trainer",report_to="wandb" ,logging_strategy = "epoch",evaluation_strategy="epoch",learning_rate = learning_rate,num_train_epochs = num_epochs,lr_scheduler_type =lr_scheduler_type, adam_beta1 = adam_beta1,adam_beta2 =adam_beta2  )


def run_trainer(model_name,out_dir,epoch,batch_size,df_train,df_valid,):
  df_train = df_train[['text','label']].set_index('text')
  df_valid = df_valid[['text','label']].set_index('text')
  #display(df_train)
  #display(df_valid)

  dataset = ds.dataset(pa.Table.from_pandas(df_valid).to_batches())
  ### convert to Huggingface dataset
  validation_dataset_torch = Dataset(pa.Table.from_pandas(df_valid))

  dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())
  ### convert to Huggingface dataset
  training_dataset_torch = Dataset(pa.Table.from_pandas(df_train))

  #print(training_dataset_torch)
  #print(validation_dataset_torch)

  from transformers import AutoTokenizer

  tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)



  def tokenize_function(data):
      return tokenizer(data['text'], padding="max_length", truncation=True,)


  train_dataset = training_dataset_torch.map(tokenize_function, batched=True)
  eval_dataset = validation_dataset_torch.map(tokenize_function, batched=True)
  print(train_dataset)
  print(eval_dataset)

  import numpy as np
  from datasets import load_metric

  metric = load_metric("matthews_correlation")

  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)


  from transformers import AutoModelForSequenceClassification

  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  #model.to(device)
  args = TrainingArguments(
      #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
      #'/content/' + f"{model_name}-finetuned-",
      out_dir,
      #report_to="wandb",
      overwrite_output_dir = True,
      logging_dir = out_dir,
      evaluation_strategy = "epoch",
      logging_strategy = "epoch",
      #save_strategy = "NO",
      learning_rate=2.6510704963161386e-06,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epoch,
      weight_decay=0.01,
      metric_for_best_model='matthews_correlation',
      save_total_limit = 2,
      save_strategy = "epoch",
      load_best_model_at_end = True,
      seed = 18
      #push_to_hub=True,
  )



  trainer = Trainer(
      model,
      args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )

  #trainer = Trainer(
    # model=model,
      #args=args,
    # train_dataset=train_dataset,
    #  eval_dataset=eval_dataset,
  #   compute_metrics=compute_metrics,
  #)


    # training
  train_result = trainer.train() 
  logs = trainer.state.log_history
  # compute train results
  metrics = train_result.metrics
  max_train_samples = len(train_dataset)
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  # save train results
  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)

  # compute evaluation results
  metrics = trainer.evaluate()
  max_val_samples = len(eval_dataset)
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  # save evaluation results
  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)
  import pickle
  save_logs_dir = out_dir
  name_logs = "logs"
  
  my_file = save_logs_dir + "/" + name_logs + ".p"
  print("saving to ",my_file )

  pickle.dump( logs, open( my_file, "wb" )    ) 

  ld = pickle.load( open( my_file, "rb" ) )


  print(logs)
  print(logs[2])
  print(type(logs))

  print(ld)
  print(type(ld))
  print(logs == ld)
  
  import torch
  torch.cuda.is_available() 
  from numba import cuda
  del trainer,args
  torch.cuda.empty_cache()
  #trainer.evaluate()



In [ ]:
def run_both_training(model_name,out_dir="/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models",batch_size = 16,df_valid = df_valid):
  print("starting " ,model_name)
  out = out_dir + "/" + model_name + "/high"
 
  if not os.path.exists(out):
    
    # Create a new directory because it does not exist 
    os.makedirs(out)
  else:
    shutil.rmtree(out)

  run_trainer(model_name=model_name,out_dir = out,epoch =50,df_train = df_train_high,df_valid=df_valid,batch_size=batch_size)
  torch.cuda.empty_cache()
  print(model_name," is completed")
  #out = out_dir + "/" + model_name + "/high_then_low"
  #run_trainer(model_name=model_name,out_dir= out,epoch=50,df_train = df_train_low,df_valid=df_valid)

In [ ]:
#run_both_training("Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment")

run_both_training("cardiffnlp/bertweet-base-sentiment")
run_both_training("svalabs/twitter-xlm-roberta-bitcoin-sentiment")
run_both_training("cardiffnlp/twitter-xlm-roberta-base-sentiment")
run_both_training("amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061")
run_both_training("finiteautomata/bertweet-base-sentiment-analysis")




In [ ]:
!nvidia-smi 

In [ ]:
!kill -9 -1